In [1]:
import requests
import pandas as pd

# Define the base URL for the SEC API to fetch company financial concepts
base_url = "https://data.sec.gov/api/xbrl/companyconcept/CIK{}/us-gaap/{}.json"

# Set up headers to avoid being blocked by the SEC API
headers = {
    'User-Agent': 'youremail@hotmail.com'  # Use your own email here
}

# List of companies with their CIKs
companies = {
    'Tesla': '0001318605',
    'Microsoft': '0000789019',
    'Apple': '0000320193'
}

# Financial taxonomies you want to extract, specific to each company
company_specific_taxonomies = {
    'Tesla': ['Revenues', 'NetIncomeLoss', 'Assets', 'Liabilities', 'NetCashProvidedByUsedInOperatingActivities'],
    'Microsoft': ['RevenueFromContractWithCustomerExcludingAssessedTax', 'NetIncomeLoss', 'Assets', 'Liabilities', 'NetCashProvidedByUsedInOperatingActivities'],
    'Apple': ['RevenueFromContractWithCustomerExcludingAssessedTax', 'NetIncomeLoss', 'Assets', 'Liabilities', 'NetCashProvidedByUsedInOperatingActivities']
}

# Function to map taxonomies to a standard metric name for the final output
def map_taxonomy_to_metric(taxonomy):
    if taxonomy in ['Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax']:
        return 'Revenues'
    elif taxonomy == 'NetIncomeLoss':
        return 'Net Income'
    elif taxonomy == 'NetCashProvidedByUsedInOperatingActivities':
        return 'Cash Flow from Operating Activities'
    else:
        return taxonomy  # For Assets and Liabilities, leave them as is

# Function to fetch and filter financial data for a company and taxonomy
def fetch_financial_data(cik, taxonomy):
    url = base_url.format(cik, taxonomy)
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        # Debugging: Print out available units if they exist
        if 'units' not in data:
            print(f"No 'units' found for {taxonomy} in {cik}. Available data keys: {list(data.keys())}")
            return []
        
        # We are only interested in the USD units
        if 'USD' in data['units']:
            financial_data = []
            yearly_data = {2021: None, 2022: None, 2023: None}  # To store 10-K or fallback 10-Q data

            # Iterate through the records to find 10-K or 10-Q forms
            for record in data['units']['USD']:
                # Only process data for the fiscal years 2021, 2022, and 2023
                if record.get('fy') in [2021, 2022, 2023]:
                    form_type = record['form']
                    fy = record['fy']
                    # Prioritize 10-K forms
                    if form_type == '10-K':
                        yearly_data[fy] = record
                    # Fallback to 10-Q if 10-K isn't available yet
                    elif form_type == '10-Q' and yearly_data[fy] is None:
                        yearly_data[fy] = record
            
            # Process the stored data for each fiscal year
            for fy, record in yearly_data.items():
                if record:
                    # Convert values to millions and format with commas
                    value_in_millions = record['val'] / 1_000_000
                    formatted_value = f"{value_in_millions:,.0f}"  # Format with commas and 0 decimal places
                    financial_data.append({
                        'Metric': map_taxonomy_to_metric(taxonomy),  # Map the taxonomy to a consistent metric name
                        'Date': record['end'],
                        'Value (in millions USD)': formatted_value,
                        'Form': record['form'],
                        'Accession Number': record['accn'],
                        'Company': data['entityName']
                    })
            return financial_data
        else:
            print(f"No USD data found for {taxonomy} in {cik}")
    else:
        print(f"Failed to fetch data for CIK {cik}, taxonomy {taxonomy}: Status code {response.status_code}")
    return []

# List to store all financial data
all_financial_data = []

# Loop through each company and taxonomy, fetching and appending the data
for company, cik in companies.items():
    print(f"\nFetching financial data for {company} (CIK: {cik})")
    
    # Use the company-specific taxonomies for each company
    taxonomies = company_specific_taxonomies[company]
    
    for taxonomy in taxonomies:
        print(f"Fetching data for {taxonomy}")
        data = fetch_financial_data(cik, taxonomy)
        if data:
            all_financial_data.extend(data)

# Convert the collected data into a Pandas DataFrame
df = pd.DataFrame(all_financial_data)

# Save the data to a CSV file
df.to_csv('financial_data_10k_with_10q_fallback.csv', index=False)

# Display the DataFrame
display(df)


Fetching financial data for Tesla (CIK: 0001318605)
Fetching data for Revenues
Fetching data for NetIncomeLoss
Fetching data for Assets
Fetching data for Liabilities
Fetching data for NetCashProvidedByUsedInOperatingActivities

Fetching financial data for Microsoft (CIK: 0000789019)
Fetching data for RevenueFromContractWithCustomerExcludingAssessedTax
Fetching data for NetIncomeLoss
Fetching data for Assets
Fetching data for Liabilities
Fetching data for NetCashProvidedByUsedInOperatingActivities

Fetching financial data for Apple (CIK: 0000320193)
Fetching data for RevenueFromContractWithCustomerExcludingAssessedTax
Fetching data for NetIncomeLoss
Fetching data for Assets
Fetching data for Liabilities
Fetching data for NetCashProvidedByUsedInOperatingActivities


,Metric,Date,Value (in millions USD),Form,Accession Number,Company
0,Revenues,2021-12-31,"53,823",10-K,0000950170-22-000796,"Tesla, Inc."
1,Revenues,2022-12-31,"81,462",10-K,0000950170-23-001409,"Tesla, Inc."
2,Revenues,2023-12-31,"96,773",10-K,0001628280-24-002390,"Tesla, Inc."
3,Net Income,2021-12-31,"5,519",10-K,0000950170-22-000796,"Tesla, Inc."
4,Net Income,2022-12-31,"12,556",10-K,0000950170-23-001409,"Tesla, Inc."
5,Net Income,2023-12-31,"14,997",10-K,0001628280-24-002390,"Tesla, Inc."
6,Assets,2021-12-31,"62,131",10-K,0000950170-22-000796,"Tesla, Inc."
7,Assets,2022-12-31,"82,338",10-K,0000950170-23-001409,"Tesla, Inc."
8,Assets,2023-12-31,"106,618",10-K,0001628280-24-002390,"Tesla, Inc."
9,Liabilities,2021-12-31,"30,548",10-K,0000950170-22-000796,"Tesla, Inc."


In [2]:
# Create a copy of the df DataFrame
df_test = df.copy()

In [4]:
# Using a regular expression to remove any non-numeric characters, except periods and commas
df_test['Value (in millions USD)'] = pd.to_numeric(
    df_test['Value (in millions USD)']
    .astype(str)  # Ensure it's treated as a string
    .str.replace(r'[^\d.-]', '', regex=True),  # Keep only digits, periods, and minus signs
    errors='coerce'  # Coerce errors into NaN
)
display(df_test)

,Metric,Date,Value (in millions USD),Form,Accession Number,Company
0,Revenues,2021-12-31,53823,10-K,0000950170-22-000796,"Tesla, Inc."
1,Revenues,2022-12-31,81462,10-K,0000950170-23-001409,"Tesla, Inc."
2,Revenues,2023-12-31,96773,10-K,0001628280-24-002390,"Tesla, Inc."
3,Net Income,2021-12-31,5519,10-K,0000950170-22-000796,"Tesla, Inc."
4,Net Income,2022-12-31,12556,10-K,0000950170-23-001409,"Tesla, Inc."
5,Net Income,2023-12-31,14997,10-K,0001628280-24-002390,"Tesla, Inc."
6,Assets,2021-12-31,62131,10-K,0000950170-22-000796,"Tesla, Inc."
7,Assets,2022-12-31,82338,10-K,0000950170-23-001409,"Tesla, Inc."
8,Assets,2023-12-31,106618,10-K,0001628280-24-002390,"Tesla, Inc."
9,Liabilities,2021-12-31,30548,10-K,0000950170-22-000796,"Tesla, Inc."


In [5]:
# Creating a copy
df_numeric = df_test.copy()

In [7]:
# Calculate percentage growth for Revenues and round to four decimals
df_numeric['Revenue Growth (%)'] = df_numeric[df_numeric['Metric'] == 'Revenues'].groupby(['Company'])['Value (in millions USD)'].pct_change() * 100
df_numeric['Revenue Growth (%)'] = df_numeric['Revenue Growth (%)'].round(4)

# Calculate percentage growth for Net Income and round to four decimals
df_numeric['Net Income Growth (%)'] = df_numeric[df_numeric['Metric'] == 'Net Income'].groupby(['Company'])['Value (in millions USD)'].pct_change() * 100
df_numeric['Net Income Growth (%)'] = df_numeric['Net Income Growth (%)'].round(4)

# Calculate percentage growth for Cash Flow from Operating Activities and round to four decimals
df_numeric['Cash Flow Growth (%)'] = df_numeric[df_numeric['Metric'] == 'Cash Flow from Operating Activities'].groupby(['Company'])['Value (in millions USD)'].pct_change() * 100
df_numeric['Cash Flow Growth (%)'] = df_numeric['Cash Flow Growth (%)'].round(4)

# Calculate percentage growth for Assets and round to four decimals
df_numeric['Assets Growth (%)'] = df_numeric[df_numeric['Metric'] == 'Assets'].groupby(['Company'])['Value (in millions USD)'].pct_change() * 100
df_numeric['Assets Growth (%)'] = df_numeric['Assets Growth (%)'].round(4)

# Calculate percentage growth for Liabilities and round to four decimals
df_numeric['Liabilities Growth (%)'] = df_numeric[df_numeric['Metric'] == 'Liabilities'].groupby(['Company'])['Value (in millions USD)'].pct_change() * 100
df_numeric['Liabilities Growth (%)'] = df_numeric['Liabilities Growth (%)'].round(4)

# Display the updated DataFrame
display(df_numeric)


,Metric,Date,Value (in millions USD),Form,Accession Number,Company,Revenue Growth (%),Net Income Growth (%),Cash Flow Growth (%),Assets Growth (%),Liabilities Growth (%)
0,Revenues,2021-12-31,53823,10-K,0000950170-22-000796,"Tesla, Inc.",NaN,NaN,NaN,NaN,NaN
1,Revenues,2022-12-31,81462,10-K,0000950170-23-001409,"Tesla, Inc.",51.3517,NaN,NaN,NaN,NaN
2,Revenues,2023-12-31,96773,10-K,0001628280-24-002390,"Tesla, Inc.",18.7953,NaN,NaN,NaN,NaN
3,Net Income,2021-12-31,5519,10-K,0000950170-22-000796,"Tesla, Inc.",NaN,NaN,NaN,NaN,NaN
4,Net Income,2022-12-31,12556,10-K,0000950170-23-001409,"Tesla, Inc.",NaN,127.5050,NaN,NaN,NaN
5,Net Income,2023-12-31,14997,10-K,0001628280-24-002390,"Tesla, Inc.",NaN,19.4409,NaN,NaN,NaN
6,Assets,2021-12-31,62131,10-K,0000950170-22-000796,"Tesla, Inc.",NaN,NaN,NaN,NaN,NaN
7,Assets,2022-12-31,82338,10-K,0000950170-23-001409,"Tesla, Inc.",NaN,NaN,NaN,32.5232,NaN
8,Assets,2023-12-31,106618,10-K,0001628280-24-002390,"Tesla, Inc.",NaN,NaN,NaN,29.4882,NaN
9,Liabilities,2021-12-31,30548,10-K,0000950170-22-000796,"Tesla, Inc.",NaN,NaN,NaN,NaN,NaN


In [8]:
# Pivot the table with 'Metric' as columns and 'Date' as index
df_pivoted = df_numeric.pivot_table(index=['Date', 'Company'], 
                                 columns='Metric', 
                                 values=['Value (in millions USD)', 
                                         'Revenue Growth (%)', 
                                         'Net Income Growth (%)', 
                                         'Cash Flow Growth (%)',
                                         'Assets Growth (%)', 
                                         'Liabilities Growth (%)'])

# Reset index for a cleaner look
df_pivoted = df_pivoted.reset_index()

# Display the new pivoted DataFrame
display(df_pivoted)


Date                Company Assets Growth (%)  \
Metric                                               Assets   
0       2021-06-30  MICROSOFT CORPORATION               NaN   
1       2021-09-25             Apple Inc.               NaN   
2       2021-12-31            Tesla, Inc.               NaN   
3       2022-06-30  MICROSOFT CORPORATION            9.3059   
4       2022-09-24             Apple Inc.            0.4994   
5       2022-12-31            Tesla, Inc.           32.5232   
6       2023-06-30  MICROSOFT CORPORATION           12.9196   
7       2023-09-30             Apple Inc.           -0.0488   
8       2023-12-31            Tesla, Inc.           29.4882   

                      Cash Flow Growth (%) Liabilities Growth (%)  \
Metric Cash Flow from Operating Activities            Liabilities   
0                                      NaN                    NaN   
1                                      NaN                    NaN   
2                                      NaN                    NaN   
3                                  16.0216                 3.3928   
4                                  17.4100                 4.9220   
5                                  28.0682                19.2877   
6                                  -1.6319                 3.7595   
7                                  -9.5030                -3.8552   
8                                  -9.9701                18.0269   

       Net Income Growth (%) Revenue Growth (%) Value (in millions USD)  \
Metric            Net Income           Revenues                  Assets   
0                        NaN                NaN                333779.0   
1                        NaN                NaN                351002.0   
2                        NaN                NaN                 62131.0   
3                    18.7152            17.9561                364840.0   
4                     5.4109             7.7938                352755.0   
5                   127.5050            51.3517                 82338.0   
6                    -0.5183             6.8820                411976.0   
7                    -2.8135            -2.8005                352583.0   
8                    19.4409            18.7953                106618.0   

                                                                             
Metric Cash Flow from Operating Activities Liabilities Net Income  Revenues  
0                                  76740.0    191791.0    61271.0  168088.0  
1                                 104038.0    287912.0    94680.0  365817.0  
2                                  11497.0     30548.0     5519.0   53823.0  
3                                  89035.0    198298.0    72738.0  198270.0  
4                                 122151.0    302083.0    99803.0  394328.0  
5                                  14724.0     36440.0    12556.0   81462.0  
6                                  87582.0    205753.0    72361.0  211915.0  
7                                 110543.0    290437.0    96995.0  383285.0  
8                                  13256.0     43009.0    14997.0   96773.0

In [9]:
# Convert the 'Date' column to datetime format
df_pivoted['Date'] = pd.to_datetime(df_pivoted['Date'], errors='coerce')
df_pivoted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column                                                          Non-Null Count  Dtype         
---  ------                                                          --------------  -----         
 0   (Date, )                                                        9 non-null      datetime64[ns]
 1   (Company, )                                                     9 non-null      object        
 2   (Assets Growth (%), Assets)                                     6 non-null      float64       
 3   (Cash Flow Growth (%), Cash Flow from Operating Activities)     6 non-null      float64       
 4   (Liabilities Growth (%), Liabilities)                           6 non-null      float64       
 5   (Net Income Growth (%), Net Income)                             6 non-null      float64       
 6   (Revenue Growth (%), Revenues)                                  6 non-null      float64       

In [11]:
# Sort the pivoted DataFrame by 'Date' and 'Company'
df_pivoted_sorted = df_pivoted.sort_values(by=['Company','Date'])

# Display the sorted DataFrame
display(df_pivoted_sorted)

Date                Company Assets Growth (%)  \
Metric                                              Assets   
1      2021-09-25             Apple Inc.               NaN   
4      2022-09-24             Apple Inc.            0.4994   
7      2023-09-30             Apple Inc.           -0.0488   
0      2021-06-30  MICROSOFT CORPORATION               NaN   
3      2022-06-30  MICROSOFT CORPORATION            9.3059   
6      2023-06-30  MICROSOFT CORPORATION           12.9196   
2      2021-12-31            Tesla, Inc.               NaN   
5      2022-12-31            Tesla, Inc.           32.5232   
8      2023-12-31            Tesla, Inc.           29.4882   

                      Cash Flow Growth (%) Liabilities Growth (%)  \
Metric Cash Flow from Operating Activities            Liabilities   
1                                      NaN                    NaN   
4                                  17.4100                 4.9220   
7                                  -9.5030                -3.8552   
0                                      NaN                    NaN   
3                                  16.0216                 3.3928   
6                                  -1.6319                 3.7595   
2                                      NaN                    NaN   
5                                  28.0682                19.2877   
8                                  -9.9701                18.0269   

       Net Income Growth (%) Revenue Growth (%) Value (in millions USD)  \
Metric            Net Income           Revenues                  Assets   
1                        NaN                NaN                351002.0   
4                     5.4109             7.7938                352755.0   
7                    -2.8135            -2.8005                352583.0   
0                        NaN                NaN                333779.0   
3                    18.7152            17.9561                364840.0   
6                    -0.5183             6.8820                411976.0   
2                        NaN                NaN                 62131.0   
5                   127.5050            51.3517                 82338.0   
8                    19.4409            18.7953                106618.0   

                                                                             
Metric Cash Flow from Operating Activities Liabilities Net Income  Revenues  
1                                 104038.0    287912.0    94680.0  365817.0  
4                                 122151.0    302083.0    99803.0  394328.0  
7                                 110543.0    290437.0    96995.0  383285.0  
0                                  76740.0    191791.0    61271.0  168088.0  
3                                  89035.0    198298.0    72738.0  198270.0  
6                                  87582.0    205753.0    72361.0  211915.0  
2                                  11497.0     30548.0     5519.0   53823.0  
5                                  14724.0     36440.0    12556.0   81462.0  
8                                  13256.0     43009.0    14997.0   96773.0

In [12]:
# Group by Company and create a dictionary of DataFrames
company_dfs = {company: df_pivoted_sorted[df_pivoted_sorted['Company'] == company] for company in df_pivoted_sorted['Company'].unique()}

# Access individual DataFrames by company
tesla_df = company_dfs['Tesla, Inc.']
microsoft_df = company_dfs['MICROSOFT CORPORATION']
apple_df = company_dfs['Apple Inc.']


## Key Ratios

1. **Profitability Ratios**
   
   - **Net Profit Margin**:
  
     
     $$\text{Net Profit Margin} = \frac{\text{Net Income}}{\text{Total Revenue}} \times 100$$
     
   - **Purpose**: measures how much profit is generated from revenue. a higher percentage indicates more profitability.
  
   - **Return on Assets (ROA)**:
  
     $$\text{ROA} = \frac{\text{Net Income}}{\text{Total Assets}} \times 100$$
  
   - **Purpose**: Indicates how efficiently a company is using its assets to generate profit. A higher ROA means better efficiency.


2. **Liquidity Ratios**
    
   These measure a company’s ability to meet its short-term obligations. While **Working Capital** and **Current Ratio** are more common liquidity measures, they typically require current assets and liabilities. But with **Total Assets** and **Total Liabilities**, you can calculate:

   - **Debt to Assets Ratio**:
  
     
     $$\text{Debt to Assets Ratio} = \frac{\text{Total Liabilities}}{\text{Total Assets}} \times 100$$
     
     - **Purpose**: Shows the percentage of assets that are financed by debt. A lower ratio generally indicates a more financially stable company.
       

   - **Equity to Assets Ratio**:
  
     
     $$\text{Equity to Assets Ratio} = \frac{\text{Total Assets} - \text{Total Liabilities}}{\text{Total Assets}} \times 100$$
  
   
     
     - **Purpose**: Shows the percentage of assets financed by shareholders' equity. A higher ratio is generally a positive sign for investors.


3. **Leverage Ratios**
   
   - **Debt to Equity Ratio**:

     $$\text{Debt to Equity Ratio} = \frac{\text{Total Liabilities}}{\text{Total Assets} - \text{Total Liabilities}}$$
     
     - **Purpose**: Measures the relative proportion of debt and equity used to finance a company's assets. A higher ratio indicates more leverage and potentially higher risk.


4. **Efficiency Ratios**
   
   - **Asset Turnover Ratio**:
     
     $$\text{Asset Turnover Ratio} = \frac{\text{Total Revenue}}{\text{Total Assets}}$$
     
     - **Purpose**: Indicates how efficiently a company is using its assets to generate revenue. A higher ratio suggests better efficiency.


5. **Cash Flow Ratios**
   
   - **Operating Cash Flow to Revenue Ratio**:
  
     
     $$\text{Operating Cash Flow to Revenue Ratio} = \frac{\text{Cash Flow from Operating Activities}}{\text{Total Revenue}} \times 100$$
     
     - **Purpose**: Measures how well a company is generating cash from its revenue. A higher ratio indicates better cash flow efficiency.

   - **Operating Cash Flow to Net Income Ratio**:
  
     
     $$\text{Operating Cash Flow to Net Income Ratio} = \frac{\text{Cash Flow from Operating Activities}}{\text{Net Income}}$$
     
     - **Purpose**: Compares operating cash flow to net income to gauge the quality of earnings. If operating cash flow is consistently higher than net income, it suggests strong earnings quality.


In [13]:
def calculate_financial_ratios(
    df,
    revenue_column=('Value (in millions USD)', 'Revenues'),
    net_income_column=('Value (in millions USD)', 'Net Income'),
    assets_column=('Value (in millions USD)', 'Assets'),
    liabilities_column=('Value (in millions USD)', 'Liabilities'),
    cash_flow_column=('Value (in millions USD)', 'Cash Flow from Operating Activities')
):
    """
    Function to calculate various financial ratios.

    Args:
        df: pandas DataFrame containing financial data with MultiIndex columns.
        revenue_column: The tuple representing the column for Total Revenue.
        net_income_column: The tuple representing the column for Net Income.
        assets_column: The tuple representing the column for Total Assets.
        liabilities_column: The tuple representing the column for Total Liabilities.
        cash_flow_column: The tuple representing the column for Cash Flow from Operating Activities.

    Returns:
        DataFrame with calculated financial ratios.
    """

    # Create a copy of the DataFrame to avoid SettingWithCopyWarning
    df = df.copy()

    # Ensure necessary columns are numeric to avoid calculation issues
    df.loc[:, revenue_column] = pd.to_numeric(df.loc[:, revenue_column], errors='coerce')
    df.loc[:, net_income_column] = pd.to_numeric(df.loc[:, net_income_column], errors='coerce')
    df.loc[:, assets_column] = pd.to_numeric(df.loc[:, assets_column], errors='coerce')
    df.loc[:, liabilities_column] = pd.to_numeric(df.loc[:, liabilities_column], errors='coerce')
    df.loc[:, cash_flow_column] = pd.to_numeric(df.loc[:, cash_flow_column], errors='coerce')

    # Calculate Profitability Ratios
    df['Net Profit Margin (%)'] = (df.loc[:, net_income_column] / df.loc[:, revenue_column]) * 100
    df['Return on Assets (ROA) (%)'] = (df.loc[:, net_income_column] / df.loc[:, assets_column]) * 100

    # Calculate Liquidity Ratios
    df['Debt to Assets Ratio (%)'] = (df.loc[:, liabilities_column] / df.loc[:, assets_column]) * 100
    df['Equity to Assets Ratio (%)'] = ((df.loc[:, assets_column] - df.loc[:, liabilities_column]) / df.loc[:, assets_column]) * 100

    # Calculate Leverage Ratios
    df['Debt to Equity Ratio'] = df.loc[:, liabilities_column] / (df.loc[:, assets_column] - df.loc[:, liabilities_column])

    # Calculate Efficiency Ratios
    df['Asset Turnover Ratio'] = df.loc[:, revenue_column] / df.loc[:, assets_column]

    # Calculate Cash Flow Ratios
    df['Operating Cash Flow to Revenue Ratio (%)'] = (df.loc[:, cash_flow_column] / df.loc[:, revenue_column]) * 100
    df['Operating Cash Flow to Net Income Ratio'] = df.loc[:, cash_flow_column] / df.loc[:, net_income_column]

    # Return the DataFrame with the new columns
    result_columns = [
        'Company', 'Date', 'Net Profit Margin (%)', 'Return on Assets (ROA) (%)',
        'Debt to Assets Ratio (%)', 'Equity to Assets Ratio (%)', 'Debt to Equity Ratio',
        'Asset Turnover Ratio', 'Operating Cash Flow to Revenue Ratio (%)',
        'Operating Cash Flow to Net Income Ratio'
    ]
    return df[result_columns].copy()


In [14]:
# Running the function with Tesla DataFrame
df_ratios_tesla = calculate_financial_ratios(tesla_df)
display(df_ratios_tesla)

,Company,Date,Net Profit Margin (%),Return on Assets (ROA) (%),Debt to Assets Ratio (%),Equity to Assets Ratio (%),Debt to Equity Ratio,Asset Turnover Ratio,Operating Cash Flow to Revenue Ratio (%),Operating Cash Flow to Net Income Ratio
Metric,,,,,,,,,,
2,"Tesla, Inc.",2021-12-31,10.253981,8.882844,49.167082,50.832918,0.967229,0.866283,21.360757,2.083167
5,"Tesla, Inc.",2022-12-31,15.413322,15.249338,44.256601,55.743399,0.793934,0.989361,18.074685,1.172666
8,"Tesla, Inc.",2023-12-31,15.497091,14.066105,40.339342,59.660658,0.676146,0.907661,13.698036,0.883910


In [15]:
# Now run the function with your DataFrame
df_ratios_microsoft = calculate_financial_ratios(microsoft_df)
display(df_ratios_microsoft)

,Company,Date,Net Profit Margin (%),Return on Assets (ROA) (%),Debt to Assets Ratio (%),Equity to Assets Ratio (%),Debt to Equity Ratio,Asset Turnover Ratio,Operating Cash Flow to Revenue Ratio (%),Operating Cash Flow to Net Income Ratio
Metric,,,,,,,,,,
0,MICROSOFT CORPORATION,2021-06-30,36.451740,18.356757,57.460475,42.539525,1.350755,0.503591,45.654657,1.252469
3,MICROSOFT CORPORATION,2022-06-30,36.686337,19.936959,54.352045,45.647955,1.190679,0.543444,44.905936,1.224051
6,MICROSOFT CORPORATION,2023-06-30,34.146238,17.564373,49.942958,50.057042,0.997721,0.514387,41.328835,1.210348


In [16]:
# Running the function with Apple DataFrame
df_ratios_apple = calculate_financial_ratios(apple_df)
display(df_ratios_apple)

,Company,Date,Net Profit Margin (%),Return on Assets (ROA) (%),Debt to Assets Ratio (%),Equity to Assets Ratio (%),Debt to Equity Ratio,Asset Turnover Ratio,Operating Cash Flow to Revenue Ratio (%),Operating Cash Flow to Net Income Ratio
Metric,,,,,,,,,,
1,Apple Inc.,2021-09-25,25.881793,26.974205,82.025743,17.974257,4.563512,1.042208,28.439903,1.098838
4,Apple Inc.,2022-09-24,25.309641,28.292441,85.635356,14.364644,5.961537,1.117852,30.977004,1.223921
7,Apple Inc.,2023-09-30,25.306234,27.509835,82.374079,17.625921,4.673462,1.087077,28.840941,1.139677


# Financial Ratio Analysis Summary

## Introduction  
In this analysis, we calculated various financial ratios for three companies: Tesla, Microsoft, and Apple. The purpose of this analysis was to observe trends and patterns in profitability, liquidity, leverage, and efficiency using key financial metrics over multiple years. We used the following financial metrics:

- Total Revenue  
- Net Income  
- Total Assets  
- Total Liabilities  
- Cash Flow from Operating Activities  

### Ratios Calculated:
- **Net Profit Margin (%)**: How much net income is generated as a percentage of revenue.  
- **Return on Assets (ROA %)**: Shows how efficiently the company is using its assets to generate profit.  
- **Debt to Assets Ratio (%)**: Measures the proportion of a company's assets that are financed by debt.  
- **Equity to Assets Ratio (%)**: The percentage of a company’s assets financed by shareholders' equity.  
- **Debt to Equity Ratio**: Measures the leverage of a company by comparing total liabilities to total shareholder equity.  
- **Asset Turnover Ratio**: Shows how efficiently a company uses its assets to generate sales.  
- **Operating Cash Flow to Revenue Ratio (%)**: Indicates the percentage of revenue that is turned into operating cash flow.  
- **Operating Cash Flow to Net Income Ratio**: Shows how well net income is supported by cash flow from operating activities.  

## Company-Specific Findings

### 1. Tesla Inc.  
- **Net Profit Margin** and **Return on Assets** improved from 2021 to 2022, indicating Tesla became more profitable and efficient. However, there was a slight decline in both metrics in 2023.  
- **Debt to Assets Ratio** decreased over time, suggesting Tesla relied less on debt to finance its assets.  
- **Operating Cash Flow to Revenue Ratio** and **Operating Cash Flow to Net Income Ratio** show Tesla’s cash flow performance relative to revenue and net income. Although the ratios fluctuated, the overall trend is positive as Tesla shows improvements in generating cash from operations relative to its earnings.  

### 2. Microsoft Corporation  
- Microsoft maintained a high **Net Profit Margin** and **Return on Assets** throughout 2021 to 2023, reflecting strong profitability and asset utilization. However, a slight decline in both metrics in 2023 suggests a mild reduction in efficiency.  
- **Debt to Equity Ratio** decreased over the years, indicating that Microsoft improved its capital structure, relying more on equity than debt.  
- Microsoft maintained a strong **Operating Cash Flow to Revenue Ratio**, demonstrating strong operational cash flow generation compared to its revenue.  

### 3. Apple Inc.  
- **Net Profit Margin** and **Return on Assets** remained relatively stable from 2021 to 2023, showing that Apple is consistently efficient in its operations and profitability.  
- Apple’s **Debt to Assets Ratio** remained high, suggesting the company has been using significant leverage, though this decreased slightly in 2023.  
- **Operating Cash Flow to Revenue Ratio** improved over time, indicating that Apple is becoming more efficient at converting revenue into operating cash flow.  

## Observations & Trends

- **Profitability**: All three companies demonstrated solid profitability, with Microsoft consistently showing the highest net profit margin. Tesla showed the most dramatic improvement, doubling its net profit margin from 2021 to 2022.  
- **Leverage**: Apple and Microsoft both rely heavily on debt to finance their assets, although Microsoft’s leverage decreased over time. Tesla's debt to assets ratio consistently decreased, indicating a more conservative approach toward leverage.  
- **Efficiency**: In terms of asset turnover, Tesla and Apple showed consistent efficiency in using assets to generate revenue, with Tesla’s asset turnover slightly declining in 2023.  
- **Liquidity**: While all companies showed strong liquidity and cash flow generation, Microsoft excelled in its ability to generate cash from operating activities compared to revenue, as reflected in its higher **Operating Cash Flow to Revenue Ratio**.  

## Conclusion  
In conclusion, all three companies exhibited strong financial performance over the analyzed period. However, each company displayed different strengths:

- Tesla showed rapid improvement in profitability and efficiency.  
- Microsoft maintained high profitability with efficient asset use and strong liquidity.  
- Apple demonstrated consistent profitability but with higher leverage.  

This analysis offers valuable insights into the financial health and operational efficiency of these tech giants, helping us better understand their financial strategies and performance.
